In [2]:
# Imports
import os
import pretty_midi as pm
import torch
from torchvision import datasets, io, models, ops, transforms, utils
import numpy as np


In [3]:
# specify the directory to start from
root_dir = "maestro-v3.0.0"

files = []
# iterate through all files in the directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        # construct the full path to the file
        file_path = os.path.join(dirpath, filename)
        # add to a list of file names
        files.append(file_path)
        
print(f"There are {len(files)} readable MIDI files.")

There are 1276 readable MIDI files.


In [4]:
# Save all files to a variable so we don't have to read the files again
MIN_NOTES = 6000 # minimum number of notes in a song to train with

file_content = []
files_read = 0
for file in files:
    f = pm.PrettyMIDI(file).instruments[0].notes
    files_read += 1
    if files_read % 20 == 0:
        print(f"Read {files_read} files.")
    if len(f) >= MIN_NOTES:
        file_content.append(f)
    

Read 20 files.
Read 40 files.
Read 60 files.
Read 80 files.
Read 100 files.
Read 120 files.
Read 140 files.
Read 160 files.
Read 180 files.
Read 200 files.
Read 220 files.
Read 240 files.
Read 260 files.
Read 280 files.
Read 300 files.
Read 320 files.
Read 340 files.
Read 360 files.
Read 380 files.
Read 400 files.
Read 420 files.
Read 440 files.
Read 460 files.
Read 480 files.
Read 500 files.
Read 520 files.
Read 540 files.
Read 560 files.
Read 580 files.
Read 600 files.
Read 620 files.
Read 640 files.
Read 660 files.
Read 680 files.
Read 700 files.
Read 720 files.
Read 740 files.
Read 760 files.
Read 780 files.
Read 800 files.
Read 820 files.
Read 840 files.
Read 860 files.
Read 880 files.
Read 900 files.
Read 920 files.
Read 940 files.
Read 960 files.
Read 980 files.
Read 1000 files.
Read 1020 files.
Read 1040 files.
Read 1060 files.
Read 1080 files.
Read 1100 files.
Read 1120 files.
Read 1140 files.
Read 1160 files.
Read 1180 files.
Read 1200 files.
Read 1220 files.
Read 1240 files.

In [5]:
# RERUN THIS TO RESET THE FILE CONTENT
file_content_copy = list(file_content)
print(len(file_content_copy))

450


In [7]:
# Total number of notes per son
num_notes=[]
i = 0
while i < len(file_content_copy):
    num_notes.append(len(file_content_copy[i]))
    i += 1
    
num_notes.sort()
print(num_notes)
print(min(num_notes))
print(max(num_notes))

[6004, 6005, 6007, 6010, 6012, 6015, 6016, 6019, 6022, 6031, 6072, 6075, 6077, 6104, 6106, 6107, 6111, 6115, 6118, 6125, 6128, 6128, 6132, 6143, 6146, 6153, 6161, 6163, 6165, 6171, 6173, 6178, 6189, 6196, 6203, 6220, 6232, 6234, 6238, 6241, 6247, 6261, 6276, 6277, 6290, 6297, 6304, 6304, 6309, 6316, 6321, 6351, 6361, 6365, 6369, 6370, 6370, 6377, 6382, 6391, 6391, 6392, 6393, 6405, 6407, 6412, 6438, 6446, 6454, 6498, 6499, 6515, 6528, 6536, 6537, 6575, 6590, 6591, 6604, 6604, 6605, 6610, 6615, 6615, 6624, 6625, 6626, 6630, 6632, 6636, 6670, 6699, 6700, 6715, 6716, 6729, 6736, 6737, 6819, 6830, 6847, 6933, 6939, 6943, 6972, 7000, 7006, 7034, 7036, 7042, 7045, 7062, 7097, 7149, 7150, 7158, 7177, 7185, 7216, 7221, 7230, 7233, 7245, 7250, 7267, 7269, 7270, 7273, 7273, 7277, 7279, 7281, 7284, 7288, 7290, 7293, 7293, 7295, 7299, 7301, 7307, 7308, 7310, 7316, 7319, 7320, 7334, 7342, 7345, 7348, 7359, 7370, 7378, 7395, 7397, 7400, 7419, 7421, 7424, 7426, 7436, 7448, 7452, 7458, 7463, 7470, 751

In [8]:
# Normalize the notes to fit on our smaller piano
for notes in file_content_copy:
    pitches = []
    for note in notes:
        pitches.append(note.pitch)
    lowest_30 = np.percentile(pitches, 30)
    change = 55-lowest_30
    print(f"Lower bound: {lowest_30}, so change by {change}.")

    # normalize to 55 average
    for note in notes:
        note.pitch += change

Lower bound: 57.0, so change by -2.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 58.0, so change by -3.0.
Lower bound: 53.0, so change by 2.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 61.0, so change by -6.0.
Lower bound: 58.0, so change by -3.0.
Lower bound: 55.0, so change by 0.0.
Lower bound: 57.0, so change by -2.0.
Lower bound: 57.0, so change by -2.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 58.0, so change by -3.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 55.0, so change by 0.0.
Lower bound: 55.0, so change by 0.0.
Lower bound: 54.0, so change by 1.0.
Lower bound: 57.0, so change by -2.0.
Lower bound: 56.0, so change by -1.0.
Lower bound: 57.0, so change by -2.0.
Lower bound: 59.0, so change by -4.0.
Lower bound: 57.0, so change by -2.0.
Lower bound: 55.0, so change by 0.0.
Lower bound: 62.0, so change by -7.0.
Lower bound: 53.0, so change by 2.0.
Lower bound: 53.0, so change by 2.0.
Lower bound: 57.0, s

In [9]:
# rescale the notes to be between 36-54, 55-84 pitch
OCTAVE_SEMITONES = 12
LOWEST_NOTE = 36
HIGHEST_NOTE = 84

for notes in file_content_copy:
    for note in notes:
        if note.pitch < LOWEST_NOTE:
            while note.pitch < LOWEST_NOTE:
                note.pitch += OCTAVE_SEMITONES
        elif note.pitch > HIGHEST_NOTE:
            while note.pitch > HIGHEST_NOTE:
                note.pitch -= OCTAVE_SEMITONES

In [10]:
# find the smallest bass amount by counting notes <= 54
low_notes = []
for notes in file_content_copy:
    num_low = 0
    for note in notes:
        if note.pitch <= 54:
            num_low += 1
    low_notes.append(num_low)

print(len(low_notes))
print(min(low_notes))

high_notes = []
for notes in file_content_copy:
    num_high = 0
    for note in notes:
        if note.pitch > 54:
            num_high += 1
    high_notes.append(num_high)

print(len(high_notes))
print(min(high_notes))



450
1520
450
4212


In [22]:
device = 'cuda'
print(torch.cuda.is_available())

True


In [23]:
# split from frequencies 54 and below, 55 and above
low = torch.zeros(len(file_content_copy), min(low_notes), 49)
high = torch.zeros(len(file_content_copy), min(high_notes), 49)
lowtime = torch.zeros(len(file_content_copy), min(low_notes), 1)
hightime = torch.zeros(len(file_content_copy), min(high_notes), 1)

for i, notes in enumerate(file_content_copy):
    k = 0
    for j in range(len(notes)):
        if k == min(low_notes):
            break
            
        if notes[j].pitch < 55:
            low[i][k][int(notes[j].pitch)-36] = 1
            lowtime[i][k][0] = notes[j].start
            k += 1
    
    k = 0
    for j in range(len(notes)):  
        if k == min(high_notes):
            break
        
        if notes[j].pitch >= 55:
            high[i][k][int(notes[j].pitch)-36] = 1
            hightime[i][k][0] = notes[j].start
            k += 1
            
print(low.shape)
print(lowtime.shape)
print(high.shape)
print(hightime.shape)

torch.Size([450, 1520, 49])
torch.Size([450, 1520, 1])
torch.Size([450, 4212, 49])
torch.Size([450, 4212, 1])


In [24]:
def check_time(l_time, h_time_arr, h_notes, thresh=0.1):
    for i, (time, note) in enumerate(zip(h_time_arr, h_notes)):
        if time < l_time:
            continue
        if time - l_time > thresh:
            break
        if time - l_time < thresh:
            if i+3 < len(h_time_arr):
                a = torch.stack((note, h_notes[i+1], h_notes[i+2], h_notes[i+3]))
                return torch.unsqueeze(a, dim=0)
            else:
                return None
    note_idx = min(enumerate(h_time_arr), key=lambda x: abs(x[1]-l_time))[0]
    if note_idx+3 < len(h_time_arr):
        b = torch.stack((h_notes[note_idx], h_notes[note_idx+1], h_notes[note_idx+2], h_notes[note_idx+3]))
        return torch.unsqueeze(b, dim=0)
    else:
        return None

train = torch.tensor([]) # size should end up being (450 x >1520) x 4 x 49
target = torch.tensor([]) # size should end up being (450 x >1520) x 1 x 49

for i, song_low in enumerate(low):
    for j, one_hot_low in enumerate(song_low):
        d2_one_hot_low = torch.unsqueeze(one_hot_low, dim=0)
        d3_one_hot_low = torch.unsqueeze(d2_one_hot_low, dim=0)
        
        s_time = lowtime[i][j][0]
        melody = check_time(s_time, hightime[i], high[i])
        
        if melody == None:
            continue
        
        train = torch.cat((train, melody))        
        target = torch.cat((target, d3_one_hot_low))
    if i == 30:
        break
    print(f"song {i}, training {train.shape}")
    print(f"song {i}, target {target.shape}")

print("Done!")

song 0, training torch.Size([1520, 4, 49])
song 0, target torch.Size([1520, 1, 49])
song 1, training torch.Size([3040, 4, 49])
song 1, target torch.Size([3040, 1, 49])
song 2, training torch.Size([4560, 4, 49])
song 2, target torch.Size([4560, 1, 49])
song 3, training torch.Size([6080, 4, 49])
song 3, target torch.Size([6080, 1, 49])
song 4, training torch.Size([7600, 4, 49])
song 4, target torch.Size([7600, 1, 49])
song 5, training torch.Size([9120, 4, 49])
song 5, target torch.Size([9120, 1, 49])
song 6, training torch.Size([10640, 4, 49])
song 6, target torch.Size([10640, 1, 49])
song 7, training torch.Size([12160, 4, 49])
song 7, target torch.Size([12160, 1, 49])
song 8, training torch.Size([13680, 4, 49])
song 8, target torch.Size([13680, 1, 49])
song 9, training torch.Size([15200, 4, 49])
song 9, target torch.Size([15200, 1, 49])
song 10, training torch.Size([16720, 4, 49])
song 10, target torch.Size([16720, 1, 49])
song 11, training torch.Size([18240, 4, 49])
song 11, target tor

In [25]:
# Save the tensor to a file
torch.save(train, "train.pt")
torch.save(target, "target.pt")